### Application 1: Take Home Salary Calculator

#### Definition

In [1]:
import numpy as np
import pandas as pd

def calculate_take_home_salary(gross_salary, pension_contribution_rate=0.05, employer_match=0.04):
    """
    Calculate take-home salary after tax, NI, and pension contributions (UK 2023/24).
    Includes employer pension matching up to a limit.
    """
    if not isinstance(gross_salary, (int, float)) or gross_salary < 0:
        raise ValueError("Gross salary must be a positive number.")
    
    # Tax thresholds and rates (2023/24 UK Tax Year)
    personal_allowance = 12570
    basic_rate_limit = 50270
    higher_rate_limit = 125140

    # Adjust personal allowance for high earners
    if gross_salary > 100000:
        reduction = (gross_salary - 100000) / 2
        personal_allowance = max(0, personal_allowance - reduction)

    # Calculate pension contributions
    employee_pension = gross_salary * pension_contribution_rate
    employer_pension = gross_salary * min(employer_match, pension_contribution_rate)
    total_pension_contribution = employee_pension + employer_pension

    # Taxable income and tax calculation
    taxable_income = max(0, gross_salary - personal_allowance)
    if gross_salary > higher_rate_limit:
        tax = (
            (basic_rate_limit - personal_allowance) * 0.2 +
            (higher_rate_limit - basic_rate_limit) * 0.4 +
            (gross_salary - higher_rate_limit) * 0.45
        )
    elif gross_salary > basic_rate_limit:
        tax = (
            (basic_rate_limit - personal_allowance) * 0.2 +
            (gross_salary - basic_rate_limit) * 0.4
        )
    else:
        tax = taxable_income * 0.2

    # National Insurance calculation
    ni_threshold = 12570
    ni_basic_limit = 50270
    if gross_salary > ni_basic_limit:
        ni = (
            (ni_basic_limit - ni_threshold) * 0.12 +
            (gross_salary - ni_basic_limit) * 0.02
        )
    elif gross_salary > ni_threshold:
        ni = (gross_salary - ni_threshold) * 0.12
    else:
        ni = 0

    # Final calculations
    take_home_salary = gross_salary - tax - ni - employee_pension

    return {
        "Gross Salary": gross_salary,
        "Tax": round(tax),
        "National Insurance": round(ni),
        "Employee Pension": round(employee_pension),
        "Employer Pension": round(employer_pension),
        "Total Pension Contribution": round(total_pension_contribution),
        "Take Home Salary": round(take_home_salary),
        "Take Home Monthly": round(take_home_salary / 12)
    }


#### Calculate Here

In [2]:
# Example Usage --- Detailed Take Home Salary Calculator 
result = calculate_take_home_salary(gross_salary=40000,pension_contribution_rate=0.05, employer_match=0.04)

print(f"\n{'='*50}")
print(f"{'Take Home Salary Breakdown':^50}")
print(f"{'='*50}\n")
print(f"{'Description':<25}{'Amount (£)':>25}")
print(f"{'-'*50}")
for item, value in result.items():
    print(f"{item:<25}{value:>25,}")
print(f"\n{'='*50}")



            Take Home Salary Breakdown            

Description                             Amount (£)
--------------------------------------------------
Gross Salary                                40,000
Tax                                          5,486
National Insurance                           3,292
Employee Pension                             2,000
Employer Pension                             1,600
Total Pension Contribution                    3,600
Take Home Salary                            29,222
Take Home Monthly                            2,435



### Application 2: Saving Plan Feasibility Assessment For House Purchase

#### Definition 

In [3]:
import numpy as np
import pandas as pd
from colorama import Fore, Style, init
def calculate_target_salary_dynamic(
    current_age, 
    current_saving, 
    current_living_cost_monthly, 
    current_salary,
    pension_contribution,
    age_to_buy_house,
    house_value,
    deposit_percentage,
    ideal_promotion_interval=3,
    ideal_promotion_salary_increase=0.25,
    savings_rate=0.4,
    loan_interest_rate=0.04,
    loan_term_years=25,
):
    """
    Calculate the target salary needed to afford a house purchase.
    """
    # Validate input parameters
    if not all(isinstance(x, (int, float)) for x in [
        current_age, current_saving, current_living_cost_monthly, 
        current_salary, age_to_buy_house, house_value]):
        raise ValueError("All numeric inputs must be numbers.")
    
    if not 0 <= deposit_percentage <= 1 or not 0 <= pension_contribution <= 1:
        raise ValueError("Percentage values must be between 0 and 1.")
    
    if current_age >= age_to_buy_house:
        raise ValueError("Current age must be less than target house-buying age.")
    
    # Calculate required values
    years_until_house = age_to_buy_house - current_age
    deposit_required = house_value * deposit_percentage
    loan_amount = house_value - deposit_required
    remaining_deposit_needed = max(0, deposit_required - current_saving)

    # Mortgage payment calculation
    monthly_interest_rate = loan_interest_rate / 12
    num_months = loan_term_years * 12
    monthly_mortgage_payment = (
        loan_amount * 
        (monthly_interest_rate * (1 + monthly_interest_rate) ** num_months) / 
        ((1 + monthly_interest_rate) ** num_months - 1)
    )
    annual_mortgage_payment = monthly_mortgage_payment * 12

    print(f"[INFO] Financial Summary:")
    print(f"       - Years until house purchase: {years_until_house} years")
    print(f"       - Total deposit required: £{round(deposit_required):,}")
    print(f"       - Remaining deposit to save: £{round(remaining_deposit_needed):,}")
    print(f"       - Annual mortgage payment after purchase: £{round(annual_mortgage_payment):,}\n")

    # Progression of savings and salary
    intervals = [(current_age + i, min(current_age + i + ideal_promotion_interval, age_to_buy_house)) 
                 for i in range(0, years_until_house, ideal_promotion_interval)]
    results = []
    target_salary = current_salary 
    errors = []
    
    for idx, (start_age, end_age) in enumerate(intervals):
        years_in_interval = end_age - start_age
        savings_needed = remaining_deposit_needed / years_until_house
        yearly_expenses = current_living_cost_monthly * 12 + (annual_mortgage_payment if end_age >= age_to_buy_house else 0)
        
        if idx > 0:
            target_salary *= (1 + ideal_promotion_salary_increase)
        else:
            target_salary = max(target_salary, yearly_expenses / savings_rate)
        
        max_savings = target_salary * savings_rate * years_in_interval
        shortfall = max(0, savings_needed * years_in_interval - max_savings)
        errors.append(shortfall)
        print('-----------')
        print(f"[INFO] Interval {idx + 1}: Age {start_age} to {end_age}")
        print(f"       - Annual Target Salary: £{round(target_salary):,}")
        print(f"       - Savings Needed: £{round(savings_needed * years_in_interval):,}")
        print(f"       - Maximum Savings Achievable: £{round(max_savings):,}")
        if shortfall > 0:
            print(f"[WARNING] Shortfall in Savings: £{round(shortfall):,}\n")
        else:
            print(f"[SUCCESS] Savings Goal Achievable!\n")

        results.append({
            "Start Age": start_age,
            "End Age": end_age,
            "Target Salary (£)": round(target_salary),
            "Savings Shortfall (£)": round(shortfall)
        })
    
    total_shortfall = sum(errors)
    print('############################## Assessment Result ############################## ')
    if total_shortfall > 0:
        print(f"[WARNING] Unable to afford the house with current plan. Total shortfall: £{round(total_shortfall):,}")
        print("[INFO] Consider increasing your savings rate or adjusting your goals.\n")
        return pd.DataFrame(results)

    print(f"[SUCCESS] Congratulations! You can afford the house by age {age_to_buy_house}.")
    print(f"[INFO] Final Target Salary at Age {age_to_buy_house}: £{round(results[-1]['Target Salary (£)']):,}\n")

    return pd.DataFrame(results)


#### Calculate Here

In [4]:
# Example Usage
if __name__ == "__main__":
    df = calculate_target_salary_dynamic(
        current_age=27,
        current_saving=30000,
        current_living_cost_monthly=1000,
        current_salary=40000,
        pension_contribution=0.05,
        age_to_buy_house=33,
        house_value=300000,
        deposit_percentage=0.4,
        ideal_promotion_interval=2,
    )
    if df is not None:
        print("\n--- Detailed Report ---")
        print(df.to_string(index=False))

[INFO] Financial Summary:
       - Years until house purchase: 6 years
       - Total deposit required: £120,000
       - Remaining deposit to save: £90,000
       - Annual mortgage payment after purchase: £11,401

-----------
[INFO] Interval 1: Age 27 to 29
       - Annual Target Salary: £40,000
       - Savings Needed: £30,000
       - Maximum Savings Achievable: £32,000
[SUCCESS] Savings Goal Achievable!

-----------
[INFO] Interval 2: Age 29 to 31
       - Annual Target Salary: £50,000
       - Savings Needed: £30,000
       - Maximum Savings Achievable: £40,000
[SUCCESS] Savings Goal Achievable!

-----------
[INFO] Interval 3: Age 31 to 33
       - Annual Target Salary: £62,500
       - Savings Needed: £30,000
       - Maximum Savings Achievable: £50,000
[SUCCESS] Savings Goal Achievable!

############################## Assessment Result ############################## 
[SUCCESS] Congratulations! You can afford the house by age 33.
[INFO] Final Target Salary at Age 33: £62,500


--